In [1]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine

In [2]:
class database_connection():
    def __init__(self, host, name, port):
        self.host = str(host)
        self.name = str(name)
        self.port = str(port)
        self.connection = "host='%s' dbname='%s'  port='%s'" % (self.host,self.name,self.port)
        
    def create(self):
        connie = "host= '" + self.host + "' port='" + self.port + "'"
        con = psycopg2.connect(connie)
        con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cur = con.cursor()
        cur.execute('CREATE DATABASE ' + self.name)
        con.close()
        
    def insert(self, query):
        con = psycopg2.connect(self.connection)   
        cur = con.cursor()
        cur.execute(query)
        con.commit()
        con.close()     
        
    def query(self, query):
        con = psycopg2.connect(self.connection)   
        cur = con.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        cols = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data =rows, columns = cols)
        con.close()
        return df
    
    def upload(self, df, dfname):
        engine = create_engine('postgresql://@' + self.host + ":" + self.port + "/" + self.name)
        df.to_sql(dfname, engine)
        
    def drop(self, dfname):
        con = psycopg2.connect(self.connection)   
        cur = con.cursor()
        cur.execute("DROP TABLE " + dfname + ";")
        con.commit()
        con.close()
    
    def clear(self):
        connie = "host= '" + self.host + "' port='" + self.port + "'"
        con = psycopg2.connect(connie)
        con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cur = con.cursor()
        cur.execute('''DROP DATABASE ''' + self.name)
        con.close()

In [4]:
systems = pd.read_json('./systems_populated.json')
stations = pd.read_json('./stations.json')
listings = pd.read_csv('./listings.csv')
commodities = pd.read_json('./commodities.json')
factions = pd.read_csv('./factions.csv')

In [24]:
elite = database_connection(host='localhost', name='elitedang', port=5400)
#elite.create()

In [5]:
def time_series(start_sys, start_stn, radius):
    time_series = pd.DataFrame(columns=['item', 'commod', 'sell_price', 'demand', 'system', 'ly', 'station', 'ls', 'faction', 'allegiance', 'state', 'margin', 'supply'])
    for day in range(1, 32):
        sysday = pd.read_json('./time-data/Day' + str(day) + '/systems_populated.json')
        stnday = pd.read_json('./time-data/Day' + str(day) + '/stations.json')
        fctday = pd.read_csv('./time-data/Day' + str(day) + '/factions.csv')
        lstday = pd.read_csv('./time-data/Day' + str(day) + '/listings.csv')
        cmdday = pd.read_json('./time-data/Day' + str(day) + '/commodities.json')
        time_series.loc[day] = profit_pipeline(start_sys, start_stn, sysday, stnday, fctday, lstday, cmdday, table=day, radius=radius, verbose=False)
        print("Day " + str(day) + " calculated")
    return time_series

In [6]:
def start_input():
    sys_enter = input("Please enter a system: ")
    sys_id = 0
    while sys_id == 0:
        for i in systems.index:
            if systems.loc[i, 'name'] == sys_enter:
                sys_id = systems.loc[i, 'id']
        if sys_id == 0:
            sys_enter = input("System not found, please enter a different system: ")
    sys_stn = stations[(stations['system_id'] == sys_id) & (stations['has_market'] == 1)]['name']
    print("List of stations in " + sys_enter + ":")
    for stn in sys_stn:
            print(stn)
    stn_enter = input("Please pick a station: ")
    stn_id = 0
    while stn_id == 0:
        if stn_enter in list(sys_stn):
            for i in stations[stations['system_id'] == sys_id].index:
                if stations.loc[i, 'name'] == stn_enter:
                    stn_id = stations.loc[i, 'id']
        else:
            stn_enter = input("Station not found, please select from the above stations: ")
    return sys_id, stn_id
#requests input so that the base station can be found so that we have a spot to start from. Since station
#names can be duplicated, we first ask for the system, as no system has two of the same stations and no two
#systems have an identical names, we can find a specific station. The list just makes stations handy to find.

In [7]:
def build_radius(sys_id, base_sys, radius=None):
    if radius == None:
        rad_enter = input("Please enter a radius (in light-years) to search: ")
        while (type(rad_enter) != int) & (type(rad_enter) != float):
            try:
                radius = int(rad_enter)
                break
            except:
                try:
                    radius = float(rad_enter)
                    break
                except:
                    rad_enter = input("That is not a number. Please enter a numeric value for the search radius: ")
    sys_radius = base_sys.copy()
    sys_radius['distance'] = 0
    base_index = sys_radius[sys_radius['id'] == sys_id].index[0]
    base_x = sys_radius.loc[base_index, 'x']
    base_y = sys_radius.loc[base_index, 'y']
    base_z = sys_radius.loc[base_index, 'z']
    for i in sys_radius.index:
        x = base_x - (sys_radius.loc[i, 'x'])
        y = base_y - (sys_radius.loc[i, 'y'])
        z = base_z - (sys_radius.loc[i, 'z'])
        sys_radius.loc[i, 'distance'] = np.sqrt(x**2 + y**2 + z**2)
    return sys_radius[sys_radius['distance'] < radius]
#from the base system we originated, this builds a dataframe of all the systems within a certain radius of the 
#base system, so that we can eventually aggregate all the stations in that radius, and all the listings from those
#systems. This allows us to save computational speed, as we don't have to go through the entire galaxy, with systems
#that may be hundreds of light-years away and take significant effort to get through, undermining efficiency.

In [8]:
def aggregate_stations(df_systems, base_stations):
    #sys_list = list(df_systems['id'])
    sys_list = list(df_systems.index)
    drop_stn = []
    for i in base_stations.index:
        if base_stations.loc[i, 'system_id'] not in sys_list:
            drop_stn.append(i)
    return base_stations.drop(drop_stn)
#takes our list of systems and gathers all the stations that are in our radius, so that we can more quickly make
#comparisons for listings, to find the station and commodity with the greatest profit value. 

In [9]:
def aggregate_listings(df_stations, base_listings):
    #stn_list = list(df_stations['id'])
    stn_list = list(df_stations.index)
    drop_lst = []
    for i in base_listings.index:
        if base_listings.loc[i, 'station_id'] not in stn_list:
            drop_lst.append(i)
    return base_listings.drop(drop_lst)
#does the same thing we did for the stations, only for the listings that are at that station.

In [10]:
#cleans up the dataframe for uploading into a SQL database, which we'll use to quick query across dataframes.
#this is the same for all the following functions. For reasons why I did what I did, check the other notebook.
def clean_systems(df_systems):
    new_systems = df_systems.copy()
    new_systems.drop(['allegiance', 'allegiance_id', 'government', 'government_id', 'state', 'state_id'], axis=1, inplace=True)
    new_systems.drop(['power', 'power_state', 'power_state_id'], axis=1, inplace=True)
    new_systems.drop(['edsm_id', 'simbad_ref', 'updated_at'], axis=1, inplace=True)
    new_systems.drop(['is_populated', 'population', 'reserve_type', 'reserve_type_id'], axis=1, inplace=True)
    for i in new_systems[new_systems['controlling_minor_faction_id'].isnull()].index:
        if new_systems.loc[i, 'name'] in ['LHS 3447', 'Shinrarta Dezhra', 'Asellus Primus']:
            new_systems.loc[i, 'controlling_minor_faction'] = 'Pilots Federation Local Branch'
            new_systems.loc[i, 'controlling_minor_faction_id'] = factions.loc[factions[factions['name'] == 'Pilots Federation Local Branch'].index[0], 'id']
    new_systems['controlling_minor_faction_inf'] = 99.0
    new_systems['secondary_minor_faction_id'] = 0
    new_systems['secondary_minor_faction_inf'] = 1.0
    for i in new_systems.index:
        faction_inf = []
        for fct in new_systems.loc[i, 'minor_faction_presences']:
            if fct['influence'] != None:
                faction_inf.append(fct['influence'])
        faction_inf.sort(reverse=True)
        if faction_inf:
            new_systems.loc[i, 'controlling_minor_faction_inf'] = faction_inf[0]
        else:
            new_systems.loc[i, 'controlling_minor_faction_inf'] = 100
        for fct in new_systems.loc[i, 'minor_faction_presences']:
            if i in new_systems[new_systems['controlling_minor_faction_id'].isnull()].index:
                if fct['influence'] == faction_inf[0]:
                    control_id = fct['minor_faction_id']
                    new_systems.loc[i, 'controlling_minor_faction_id'] = control_id
                    new_systems.loc[i, 'controlling_minor_faction'] = factions.loc[factions[factions['id'] == control_id].index[0], 'name']
            if len(faction_inf) > 1:
                if fct['influence'] == faction_inf[1]:
                    new_systems.loc[i, 'secondary_minor_faction_inf'] = fct['influence']
                    new_systems.loc[i, 'secondary_minor_faction_id'] = fct['minor_faction_id']
            else:
                new_systems.loc[i, 'secondary_minor_faction_inf'] = 0
                new_systems.loc[i, 'secondary_minor_faction_id'] = 0
    new_systems.drop('minor_faction_presences', axis=1, inplace=True)
    new_systems.dropna(inplace=True)
    new_systems['controlling_minor_faction_id'] = new_systems['controlling_minor_faction_id'].astype(int)
    new_systems['primary_economy_id'] = new_systems['primary_economy_id'].astype(int)
    new_systems['security_id'] = new_systems['security_id'].astype(int)
    new_systems.set_index('id', inplace=True)
    return new_systems

In [11]:
def clean_stations(df_stations):
    new_stations = df_stations.copy()
    new_stations.drop(['allegiance', 'allegiance_id', 'government', 'government_id', 'state', 'state_id'], axis=1, inplace=True)
    new_stations.drop(['import_commodities', 'export_commodities', 'has_docking', 'has_market'], axis=1, inplace=True)
    new_stations.drop(['updated_at', 'market_updated_at', 'outfitting_updated_at', 'shipyard_updated_at'], axis=1, inplace=True)
    new_stations.drop(['prohibited_commodities', 'selling_modules', 'selling_ships'], axis=1, inplace=True)
    new_stations.drop(['settlement_size', 'settlement_size_id', 'settlement_security', 'settlement_security_id'], axis=1, inplace=True)
    econ_types = set()
    for i in new_stations.index:
        for econ in new_stations.loc[i, 'economies']:
            econ_types.add(econ)
    for etyp in econ_types:
        new_stations['econ_' + etyp.lower()] = False
        for i in new_stations.index:
            econ = new_stations.loc[i, 'economies']
            if etyp in econ:
                new_stations.loc[i, ('econ_' + etyp.lower())] = True
    new_stations.rename(columns={'econ_high tech':'econ_high_tech'}, inplace=True)
    new_stations.drop('economies', axis=1, inplace=True)
    unk_body_list = list(new_stations[new_stations['distance_to_star'].isnull() & (new_stations['body_id'].isnull() != 1)]['body_id'])
    k_body_list = list(new_stations[(new_stations['distance_to_star'].isnull() != 1) & (new_stations['body_id'].isnull() != 1)]['body_id'])
    for body in unk_body_list:
        if body in k_body_list:
            dist_to_star = int(new_stations[(new_stations['distance_to_star'].isnull() != 1) & (new_stations['body_id'] == body)]['distance_to_star'].mean())
            for i in new_stations[new_stations['distance_to_star'].isnull() & new_stations['body_id'] == body].index:
                new_stations.loc[i, 'distance_to_star'] = dist_to_star
    new_stations.dropna(subset=['distance_to_star', 'type'], inplace=True)
    new_stations['distance_to_star'] = new_stations['distance_to_star'].astype(int)
    new_stations['type_id'] = new_stations['type_id'].astype(int)
    new_stations.drop('body_id', axis=1, inplace=True)
    for i in new_stations[new_stations['max_landing_pad_size'].isnull()].index:
        pad_stn_type = new_stations.loc[i, 'type']
        if len(new_stations[(new_stations['type'] == pad_stn_type) & (new_stations['max_landing_pad_size'].isnull() != 1)]['max_landing_pad_size'].unique()) == 1:
            new_stations.loc[i, 'max_landing_pad_size'] = new_stations[(new_stations['type'] == pad_stn_type) & (new_stations['max_landing_pad_size'].isnull() != 1)]['max_landing_pad_size'].unique()[0]
    new_stations.dropna(subset=['max_landing_pad_size'], inplace=True)
    for i in new_stations[new_stations['controlling_minor_faction_id'].isnull()].index:
        stn_sys_id = new_stations.loc[i, 'system_id']
        new_stations.loc[i, 'controlling_minor_faction_id'] = systems.loc[systems[systems['id'] == stn_sys_id].index[0], 'controlling_minor_faction_id']
    new_stations.dropna(inplace=True)
    new_stations['controlling_minor_faction_id'] = new_stations['controlling_minor_faction_id'].astype(int)
    new_stations.set_index('id', inplace=True)
    return new_stations

In [12]:
def clean_listings(df_listings):
    new_listings = df_listings.copy()
    new_listings.drop(['supply_bracket', 'demand_bracket', 'collected_at'], axis=1, inplace=True)
    new_listings.dropna(inplace=True)
    for col in new_listings.columns:
        new_listings[col] = new_listings[col].astype(int)
    new_listings.set_index('id', inplace=True)
    return new_listings

In [13]:
def clean_commodities(df_commodities):
    new_commod = df_commodities.copy()
    for i in new_commod.index:
        new_commod.loc[i, 'category'] = new_commod.loc[i, 'category']['name']
    new_commod.drop(['ed_id', 'max_buy_price', 'max_sell_price', 'min_buy_price', 'min_sell_price'], axis=1, inplace=True)
    new_commod.dropna(inplace=True)
    new_commod['average_price'] = new_commod['average_price'].astype(int)
    new_commod['buy_price_lower_average'] = new_commod['buy_price_lower_average'].astype(int)
    new_commod['sell_price_upper_average'] = new_commod['sell_price_upper_average'].astype(int)
    new_commod['category_id'] = new_commod['category_id'].astype(int)
    new_commod['is_rare'] = new_commod['is_rare'].astype(bool)
    new_commod['is_non_marketable'] = new_commod['is_non_marketable'].astype(bool)
    new_commod.set_index('id', inplace=True)
    return new_commod

In [14]:
def clean_factions(df_factions):
    new_factions = df_factions.copy()
    new_factions.drop(['updated_at', 'home_system_id'], axis=1, inplace=True)
    new_factions.loc[new_factions[new_factions['name'].isnull()].index[0], 'name'] = 'Null'
    for col in ['allegiance', 'allegiance_id','government', 'government_id']:
        for i in new_factions[new_factions[col].isnull()].index:
            if new_factions.loc[i, 'id'] in set(systems['controlling_minor_faction_id']):
                    new_factions.loc[i, col] = systems.loc[systems[systems['controlling_minor_faction_id'] == new_factions.loc[i, 'id']].index[0], col]
            elif new_factions.loc[i, 'id'] in set(stations['controlling_minor_faction_id']):
                    new_factions.loc[i, col] = stations.loc[stations[stations['controlling_minor_faction_id'] == new_factions.loc[i, 'id']].index[0], col]
    new_factions['state'].fillna('None', inplace=True)
    new_factions['state_id'].fillna(80, inplace=True)
    new_factions.dropna(inplace=True)
    new_factions['government_id'] = new_factions['government_id'].astype(int)
    new_factions['allegiance_id'] = new_factions['allegiance_id'].astype(int)
    new_factions['state_id'] = new_factions['state_id'].astype(int)
    new_factions['is_player_faction'] = new_factions['is_player_faction'].astype(bool)
    new_factions.set_index('id', inplace=True)
    return new_factions

In [15]:
def drop_listless(df_listings, df_systems, df_stations):
    stn_drop = []
    sys_drop = []
    for i in df_stations.index:
        if i not in df_listings['station_id'].unique():
            stn_drop.append(i)
    df_stations.drop(stn_drop, inplace=True)
    for i in df_systems.index:
        if i not in df_stations['system_id'].unique():
            sys_drop.append(i)
    df_systems.drop(sys_drop, inplace=True)

In [16]:
def get_prohibitions(df_stations):
    prohib = pd.DataFrame()
    prohib['id'] = df_stations['id']
    prohib['prohibited'] = df_stations['prohibited_commodities']
    return prohib

In [17]:
def build_database(db, df_systems, df_stations, df_factions, df_commodities, df_listings, table):
    db.upload(df_systems, "systems" + str(table))
    db.upload(df_stations, "stations" + str(table))
    db.upload(df_factions, "factions" + str(table))
    db.upload(df_commodities, "commodities" + str(table))
    db.upload(df_listings, "listings" + str(table))
    return db

In [18]:
def get_profit(start_stn, sql_db, table):
    df_stn = sql_db.query("""SELECT c.id as commod, supply, buy_price FROM listings""" + str(table) + """ 
                             INNER JOIN commodities""" + str(table) + """ c ON commodity_id = c.id
                             WHERE station_id = """ + str(start_stn) + """AND buy_price != 0""")
    df_tgt = sql_db.query("""SELECT c.name as item, c.id as commod, sell_price, demand,
                             sys.name as system, sys.distance as ly, stn.name as station, 
                             stn.distance_to_star as ls, f.name as faction, f.allegiance, f.state FROM listings""" + str(table) + """ 
                             INNER JOIN commodities""" + str(table) + """ c ON commodity_id = c.id
                             INNER JOIN stations""" + str(table) + """ stn ON station_id = stn.id
                             INNER JOIN systems""" + str(table) + """ sys ON stn.system_id = sys.id
                             INNER JOIN factions""" + str(table) + """ f ON stn.controlling_minor_faction_id = f.id
                             WHERE station_id !=""" + str(start_stn))
    profit_margin = pd.DataFrame(columns=['index', 'margin'])
    for c in df_stn['commod']:
        max_sell = df_tgt[df_tgt['commod'] == c]['sell_price'].max()
        df_m_sell = df_tgt[(df_tgt['sell_price'] == max_sell) & (df_tgt['commod'] == c)]
        prof_marg_ind = efficient_margin_index(df_m_sell)
        profit_margin.loc[c, 'index'] = prof_marg_ind
        profit_margin.loc[c, 'margin'] = max_sell - df_stn.loc[df_stn[df_stn['commod'] == c].index[0], 'buy_price']
    max_margin = profit_margin[profit_margin['margin'] == profit_margin['margin'].max()]
    max_margin['ly'] = df_tgt.loc[max_margin['index'], 'ly']
    max_margin['ls'] = df_tgt.loc[max_margin['index'], 'ls']
    hi_prf_mrg_ind = max_margin.loc[efficient_margin_index(max_margin), 'index']
    target_data = df_tgt.loc[hi_prf_mrg_ind]
    target_data['margin'] = max_margin.loc[efficient_margin_index(max_margin), 'margin']
    target_data['supply'] = df_stn.loc[df_stn[df_stn['commod'] == efficient_margin_index(max_margin)].index[0], 'supply']
    return target_data

In [19]:
def efficient_margin_index(df_max_sell):
    if len(df_max_sell) == 1:
        return df_max_sell.index[0]
    else:
        df_min_ly = df_max_sell[df_max_sell['ly'] == df_max_sell['ly'].min()]
        if len(df_min_ly) == 1:
            return df_min_ly.index[0]
        else:
            df_min_ls = df_min_ly[df_min_ly['ls'] == df_min_ly['ls'].min()]
            return df_min_ls.index[0]

In [20]:
def print_profit_info(target_data):
    print("Commodity: " + target_data['item'])
    print("Supply: " + str(target_data['supply']) + " / Demand: " + str(target_data['demand']))
    print("Margin: " + str(target_data['margin']))
    print("Destination: " + target_data['station'] + " in " + target_data['system'])
    print("Distance: " + str(target_data['ly']) + " ly to system, " + str(target_data['ls']) + " ls to station")
    print("Control: " + target_data['faction'] + " in a " + target_data['state'] + " state")
    print("Alignment: " + target_data['allegiance'])

In [21]:
def profit_pipeline(start_sys, start_stn, base_sys, base_stn, base_fct, base_lst, base_cmd, table=0, radius=None, verbose=True):
    sys = build_radius(start_sys, base_sys, radius)
    if verbose:
        print("Gathering nearby systems...")
    cf = clean_factions(base_fct)
    c_sys = clean_systems(sys)
    agg_stn = aggregate_stations(c_sys, base_stn)
    if verbose:
        print("Checking nearby stations...")
    c_stn = clean_stations(agg_stn)
    lst = aggregate_listings(c_stn, base_lst)
    if verbose:
        print("Gathering market listings...")
    cl = clean_listings(lst)
    cc = clean_commodities(base_cmd)
    drop_listless(cl, c_sys, c_stn)
    if verbose:
        print("Building database...")
    query = build_database(elite, c_sys, c_stn, cf, cc, cl, table)
    if verbose:
        print("Optimizing...")
    profit = get_profit(start_stn, query, table)
    if verbose:
        print("Done!")
    return profit

In [22]:
def clear_query(db, table):
    db.drop("systems" + str(table))
    db.drop("stations" + str(table))
    db.drop("factions" + str(table))
    db.drop("listings" + str(table))
    db.drop("commodities" + str(table))

In [30]:
clear_query(elite, 0)
start_pt = start_input()
profit = profit_pipeline(start_pt[0], start_pt[1], systems, stations, factions, listings, commodities)
print_profit_info(profit)

Please enter a system: Sol
List of stations in Sol:
Abraham Lincoln
Galileo
Li Qing Jao
M.Gorbachev
Daedalus
Mars High
Columbus
Titan City
Burnell Station
Ehrlich City
Haberlandt Survey
Furukawa Enterprise
Walz Depot
Durrance Camp
Schottky Reformatory
Please pick a station: Mars High
Please enter a radius (in light-years) to search: 15
Gathering nearby systems...
Checking nearby stations...
Gathering market listings...
Building database...
Optimizing...
Done!
Commodity: Biowaste
Supply: 14863 / Demand: 11849098
Margin: 605
Destination: Davy Dock in Procyon
Distance: 11.4078337941522 ly to system, 9986 ls to station
Control: Sirius Corporation in a Expansion state
Alignment: Independent


/Users/arpariseau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/arpariseau/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
/Users/arpariseau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
wolf_906 = time_series(start_pt[0], start_pt[1], 30)

/Users/arpariseau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/arpariseau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/arpariseau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop

Day 1 calculated
Day 2 calculated
Day 3 calculated
Day 4 calculated
Day 5 calculated
Day 6 calculated
Day 7 calculated
Day 8 calculated
Day 9 calculated
Day 10 calculated
Day 11 calculated
Day 12 calculated
Day 13 calculated
Day 14 calculated
Day 15 calculated
Day 16 calculated
Day 17 calculated
Day 18 calculated
Day 19 calculated
Day 20 calculated
Day 21 calculated
Day 22 calculated
Day 23 calculated
Day 24 calculated
Day 25 calculated
Day 26 calculated
Day 27 calculated
Day 28 calculated
Day 29 calculated
Day 30 calculated
Day 31 calculated


In [26]:
connie = "host= 'localhost' port='5400'"
con = psycopg2.connect(connie)
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = con.cursor()
cur.execute('''DROP DATABASE EliteDang''')
con.close()

In [76]:
start_input()

Please enter a system: Kamadhenu
List of stations in Kamadhenu:
Shajn Market
Couper Hub
Please pick a station: Shajn Market


(10350, 14748)

In [57]:
    df_stn = elite.query("""SELECT c.id as commod, supply, buy_price FROM listings11
                             INNER JOIN commodities11 c ON commodity_id = c.id
                             WHERE station_id = 2832 AND buy_price != 0""")
    df_tgt = elite.query("""SELECT c.name as item, c.id as commod, sell_price, demand,
                             sys.name as system, sys.distance as ly, stn.name as station, 
                             stn.distance_to_star as ls, f.name as faction, f.allegiance, f.state FROM listings12 
                             INNER JOIN commodities12 c ON commodity_id = c.id
                             INNER JOIN stations12 stn ON station_id = stn.id
                             INNER JOIN systems12 sys ON stn.system_id = sys.id
                             INNER JOIN factions12 f ON stn.controlling_minor_faction_id = f.id
                             WHERE station_id != 2832""")

In [58]:
df_stn

,commod,supply,buy_price
0,2,969095,85
1,3,41095,214
2,271,1624689,26
3,12,33888,241
4,8,47418,186
5,11,13512,4209
6,13,3395581,49
7,14,80871,1086
8,15,82879,1086
9,16,105690,317


In [131]:
profit_margin = pd.DataFrame(columns=['index', 'margin'])
for c in df_stn['commod']:
    max_sell = df_tgt[df_tgt['commod'] == c]['sell_price'].max()
    df_m_sell = df_tgt[(df_tgt['sell_price'] == max_sell) & (df_tgt['commod'] == c)]
    prof_marg_ind = efficient_margin_index(df_m_sell)
    profit_margin.loc[c, 'index'] = prof_marg_ind
    profit_margin.loc[c, 'margin'] = max_sell - df_stn.loc[df_stn[df_stn['commod'] == c].index[0], 'buy_price']
max_margin = profit_margin[profit_margin['margin'] == profit_margin['margin'].max()]
max_margin['ly'] = df_tgt.loc[max_margin['index'], 'ly']
max_margin['ls'] = df_tgt.loc[max_margin['index'], 'ls']
hi_prf_mrg_ind = max_margin.loc[efficient_margin_index(max_margin), 'index']
supply = df_stn.loc[df_stn[df_stn['commod'] == efficient_margin_index(max_margin)].index[0], 'supply']
print_profit_info(df_tgt.loc[hi_prf_mrg_ind], max_margin.loc[efficient_margin_index(max_margin), 'margin'], supply)

,item,commod,sell_price,demand,system,ly,station,ls,faction,allegiance,state
1099,Tobacco,11,5822,1212,Omicron-2 Eridani,19.271244,Hodgkinson Station,225727,Confederacy of Omicron-2 Eridani,Federation,War
3367,Tobacco,11,5822,844,Ross 128,19.751904,Warren Prison Mine,19,LHS 2541 Alliance Combine,Alliance,War
7852,Tobacco,11,5822,954,Omicron-2 Eridani,19.271244,Reed City,225725,Confederacy of Omicron-2 Eridani,Federation,War
13029,Tobacco,11,5822,1722,Omicron-2 Eridani,19.271244,Polansky Barracks,106,Confederacy of Omicron-2 Eridani,Federation,War


In [112]:
cl[cl['station_id'] == 2832]

,station_id,commodity_id,supply,buy_price,sell_price,demand
id,,,,,,
4661842,2832,2,1734681,85,80,0
4661843,2832,3,71307,172,145,0
4661844,2832,4,0,0,289,118340
4661845,2832,271,2775847,26,5,0
4661846,2832,5,0,0,639,375697
4661847,2832,6,0,0,6561,6561
4661848,2832,7,0,0,707,78975
4661849,2832,13,6985147,49,29,0
4661850,2832,14,145853,1086,1031,0


In [114]:
c_stn.loc[2832]

controlling_minor_faction_id                 5013
distance_to_star                              261
has_blackmarket                              True
has_commodities                              True
has_outfitting                               True
has_rearm                                    True
has_refuel                                   True
has_repair                                   True
has_shipyard                                 True
is_planetary                                False
max_landing_pad_size                            L
name                              Schneider Relay
system_id                                    4592
type                            Coriolis Starport
type_id                                         3
econ_refinery                               False
econ_colony                                 False
econ_high_tech                              False
econ_industrial                             False
econ_agriculture                             True


In [115]:
cc.loc[11]

average_price                      5056
buy_price_lower_average            4123
category                    Legal Drugs
category_id                           3
is_non_marketable                 False
is_rare                           False
name                            Tobacco
sell_price_upper_average           5857
Name: 11, dtype: object

In [165]:
clear_query(elite, 0)

In [29]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [201]:
eravate.to_csv('eravate.csv')

In [41]:
eravate = pd.read_csv('eravate.csv')

In [42]:
eravate.rename(columns={"Unnamed: 0":"day"}, inplace=True)

In [43]:
eravate.set_index('day', inplace=True)

In [44]:
eravate.tail()

,item,commod,sell_price,demand,system,ly,station,ls,faction,allegiance,state,margin,supply
day,,,,,,,,,,,,,
27,Basic Medicines,33,4673,67600,LHS 3447,4.162706,Leinster Survey,2753,LHS 3447 Cartel,Independent,Outbreak,4427,144091
28,Basic Medicines,33,4673,67600,LHS 3447,4.162706,Leinster Survey,2754,LHS 3447 Cartel,Independent,Outbreak,4427,144091
29,Basic Medicines,33,1939,3166,Frigaha,6.683848,Engle Orbital,75,Frigaha Gold Netcoms Ltd,Federation,Civil War,1693,144091
30,Basic Medicines,33,1992,2446,Kini,4.729912,Kadenyuk Orbital,14046,TitanStar Alliance,Independent,Civil War,1746,144091
31,Basic Medicines,33,1992,2446,Kini,4.729912,Kadenyuk Orbital,14045,TitanStar Alliance,Independent,Civil War,1746,144091


In [31]:
profit
#Eravate

item             Basic Medicines
commod                        33
sell_price                  1992
demand                      2446
system                      Kini
ly                       4.72991
station         Kadenyuk Orbital
ls                         14044
faction       TitanStar Alliance
allegiance           Independent
state                  Civil War
margin                      1746
supply                    144091
Name: 234, dtype: object

In [68]:
profit
#Epsilon Indi

item                            Tea
commod                           21
sell_price                     2152
demand                          747
system                     Ross 154
ly                          8.93602
station                 Elder Works
ls                             1501
faction       Labour of 70 Ophiuchi
allegiance               Federation
state                          Boom
margin                          915
supply                       151175
Name: 4018, dtype: object

In [88]:
profit
#Calennero

item                                       Gold
commod                                       42
sell_price                                11011
demand                                    21717
system                                HIP 19327
ly                                      17.5644
station                          Alfven Gateway
ls                                       330758
faction       League of HIP 19327 Justice Party
allegiance                               Empire
state                                 Civil War
margin                                     1687
supply                                       90
Name: 838, dtype: object

In [70]:
epsilon_indi

,item,commod,sell_price,demand,system,ly,station,ls,faction,allegiance,state,margin,supply
1,Water,271,661,1069,Alpha Centauri,9.636914,Hutton Orbital,6783760,Hutton Orbital Truckers Co-Operative,Independent,Boom,610,221528
2,Water,271,661,1069,Alpha Centauri,9.636914,Hutton Orbital,6783759,Hutton Orbital Truckers Co-Operative,Independent,Boom,610,221528
3,Water,271,661,1069,Alpha Centauri,9.636914,Hutton Orbital,6783758,Hutton Orbital Truckers Co-Operative,Independent,Boom,610,221528
4,Tea,21,2158,1263,Ross 154,8.936025,Birkeland City,13,Barnard's Star Advanced Corp.,Federation,Civil War,479,9718
5,Tea,21,2158,1263,Ross 154,8.936025,Birkeland City,12,Barnard's Star Advanced Corp.,Federation,Civil War,479,9718
6,Tea,21,2158,1263,Ross 154,8.936025,Birkeland City,13,Barnard's Star Advanced Corp.,Federation,Civil War,479,9718
7,Water,271,652,771,Epsilon Indi,0.000000,King Silo,143,People's Epsilon Indi Progressive Party,Federation,None,601,221528
8,Water,271,714,807,Alpha Centauri,9.636914,Hutton Orbital,6783753,Hutton Orbital Truckers Co-Operative,Independent,None,663,221528
9,Water,271,652,771,Epsilon Indi,0.000000,King Silo,143,People's Epsilon Indi Progressive Party,Federation,Civil Unrest,601,221528
10,Tea,21,2158,1263,Ross 154,8.936025,Birkeland City,13,Barnard's Star Advanced Corp.,Federation,Civil War,921,83768


In [69]:
epsilon_indi.to_csv('./epsilon_indi.csv')

In [78]:
new_listings = pd.read_csv('./time-data/Day8/listings.csv')

In [79]:
new_listings[new_listings['station_id'] == 14748]

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at


In [86]:
calennero

,item,commod,sell_price,demand,system,ly,station,ls,faction,allegiance,state,margin,supply
1,Palladium,45,14148,88105,Sauraratec,19.922923,Murakami Ring,88125,Chapterhouse of Inquisition,Empire,Boom,1604,2
2,Palladium,45,14148,88105,Sauraratec,19.922923,Murakami Ring,88127,Chapterhouse of Inquisition,Empire,Boom,1732,1
3,Palladium,45,14288,73668,Sauraratec,19.922923,Murakami Ring,88125,Chapterhouse of Inquisition,Empire,None,1872,1
4,Palladium,45,14288,73668,Sauraratec,19.922923,Murakami Ring,88120,Chapterhouse of Inquisition,Empire,Election,1872,1
5,Palladium,45,14288,73668,Sauraratec,19.922923,Murakami Ring,88115,Chapterhouse of Inquisition,Empire,Election,1872,1
6,Palladium,45,14288,73668,Sauraratec,19.922923,Murakami Ring,88115,Chapterhouse of Inquisition,Empire,Election,1744,1
7,Palladium,45,14283,63063,Sauraratec,19.922923,Murakami Ring,88117,Chapterhouse of Inquisition,Empire,None,1739,2
8,Palladium,45,14146,408468,Wayutabal,18.123437,Babakin Port,1114,Wayutabal Empire Pact,Empire,War,1602,2
9,Palladium,45,14145,2641,HIP 19327,17.564446,Alfven Gateway,330703,League of HIP 19327 Justice Party,Empire,War,1601,2
10,Palladium,45,14145,2641,HIP 19327,17.564446,Alfven Gateway,330703,League of HIP 19327 Justice Party,Empire,War,1729,1


In [87]:
calennero.to_csv('./calennero.csv')

In [30]:
wolf_906

,item,commod,sell_price,demand,system,ly,station,ls,faction,allegiance,state,margin,supply
1,Basic Medicines,33,2719,9365007,31 Aquilae,25.281501,Isherwood Gateway,5460,31 Aquilae Blue State Org,Federation,Boom,2396,10293
2,Synthetic Meat,20,2376,25050,Volungu,16.828118,Cabrera Horizons,18,Volungu Gang,Independent,Famine,2136,43437
3,Synthetic Meat,20,2376,25050,Volungu,16.828118,Cabrera Horizons,18,Volungu Gang,Independent,Famine,2136,43437
4,Synthetic Meat,20,2376,25050,Volungu,16.828118,Cabrera Horizons,18,Volungu Gang,Independent,Famine,2136,43437
5,Military Grade Fabrics,304,3221,9120,BD-17 6172,24.923496,Deslandres Survey,169183,BD-17 6172 Universal Industry,Empire,Boom,2267,304
6,Military Grade Fabrics,304,3214,9120,BD-17 6172,24.923496,Deslandres Survey,169182,BD-17 6172 Universal Industry,Empire,Boom,2260,304
7,Military Grade Fabrics,304,3214,9120,BD-17 6172,24.923496,Deslandres Survey,169182,BD-17 6172 Universal Industry,Empire,Boom,2260,304
8,Military Grade Fabrics,304,3214,9120,BD-17 6172,24.923496,Deslandres Survey,169182,BD-17 6172 Universal Industry,Empire,Boom,2260,304
9,Military Grade Fabrics,304,4213,6060,Gliese 875,21.656363,Alexandria Silo,418,Confederation of Gliese 875,Federation,Boom,3259,304
10,Military Grade Fabrics,304,4213,6060,Gliese 875,21.656363,Alexandria Silo,416,Confederation of Gliese 875,Federation,Boom,3590,2531


In [ ]:
wolf_906.to_csv('')